In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code')
import compute_indicators as ci

In [3]:
import numpy as np
import pandas as pd
import talib as ta
import tqdm.notebook as tqdm

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

In [6]:
import json

symbols = []
with open('../files/public/top_symbols.json') as top_symbols:
    symbols = json.load(top_symbols)

In [7]:
columns = json.load(open('../models/columns.json', 'rb'))

In [8]:
from keras.models import load_model

model = load_model('../models/daily.h5')

Using Theano backend.


In [9]:
from pickle import load

scaler = load(open('../models/scaler.pkl', 'rb'))

In [10]:
def predict(stock, symbol):
    return ci.predict_today(stock, columns, model, scaler)

In [11]:
stocks = []

for i in tqdm.tqdm(range(0, len(symbols), 100)):
    stocks.append(alpaca.get_barset(symbols[i:i+100], '1D', 400).df.stack(level=0).reset_index())

In [12]:
stocks = pd.concat(stocks)

In [13]:
stocks.columns = ['date', 'symbol', 'close', 'high', 'low', 'open', 'volume']
stocks.set_index('symbol', inplace=True)
stocks.date = pd.to_datetime(stocks.date, utc=True)

In [14]:
market = alpaca.get_barset('SPY', '1D', 400).df.stack(level=0).reset_index()
market.columns = ['date', 'symbol', 'close', 'high', 'low', 'open', 'volume']
market.set_index('symbol', inplace=True)
market.date = pd.to_datetime(market.date, utc=True)

In [15]:
open_positions = {position.symbol: position for position in alpaca.list_positions()}

In [16]:
def buy_stock(symbol, qty):
    try:
        alpaca.submit_order(
            symbol=symbol,
            side='buy',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to buy {qty} shares of {symbol}', e)  

In [17]:
def sell_stock(symbol, qty):
    try:
        alpaca.submit_order(
            symbol=symbol,
            side='sell',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to sell {qty} shares of {symbol}', e)

In [18]:
alpaca.cancel_all_orders()

stocks_by_symbol = stocks.groupby('symbol')

ci.compute(market)

for symbol in tqdm.tqdm(symbols):
    if symbol in stocks.index: 
        stock = stocks_by_symbol.get_group(symbol)
        ci.compute(stock)
        stock = stock.merge(market, on='date', suffixes=['', '_market'])
        prediction, last_close = predict(stock, symbol)
        print(symbol, prediction)
        if (prediction >= 0.5) and (symbol not in open_positions):
            qty = 1000 // last_close
            buy_stock(symbol, qty)
        elif (prediction < 0.5) and (symbol in open_positions):
            qty = open_positions[symbol].qty
            sell_stock(symbol, qty)

FTAI 0.7507702
AAPL 0.56005746
MSFT 0.53044707
CSCO 0.63572145
INTC 0.63132626
ORCL 0.61491144
EBAY 0.5877718
BAC 0.69177216
NIO 0.70098484
FB 0.5080541
SIRI 0.6636615
PAE 0
SNAP 0.56649226
UBER 0.751066
GE 0.7046932
TWTR 0.633407
VALE 0.6927519
VXX 0.65623176
F 0.62521213
KTB 0.68037426
AMAT 0.759271
QCOM 0.65490603
NFLX 0.5041535
NOK 0.65717536
PFE 0.58490735
BABA 0.5942663
V 0.4960484
ZNGA 0.6168925
ITUB 0.629565
BB 0.7002032
PBR 0.58760554
NVDA 0.4897244
SBUX 0.5316201
MU 0.75887275
GILD 0.66687375
HPQ 0.62625223
AMD 0.6020191
CMCSA 0.66078275
FCX 0.609231
HPE 0.7145245
GM 0.62260085
CHK 0.4745182
BIDU 0.6467365
MA 0.5017833
T 0.67596155
JD 0.610328
BBD 0.50698453
NKE 0.64297396
JPM 0.72446
INFY 0.6978997
KMI 0.66488165
GRPN 0.5520017
ROKU 0.74641657
TJX 0.68360394
WORK 0.7334438
DAL 0.6225596
WFC 0.64798224
IQ 0.6177272
MRVL 0.47145575
MS 0.7113769
IAA 0.562978
PINS 0.5921554
LVS 0.72421587
AUY 0.59396625
TSM 0.631922
SQ 0.5497765
JNPR 0.63419485
BSM 0.61680424
SLM 0.59822595
CRON

An error occured attempting to buy 18.0 shares of GRUB insufficient buying power
LEN 0.5848916
An error occured attempting to buy 19.0 shares of LEN insufficient buying power
KAR 0.69204
An error occured attempting to buy 84.0 shares of KAR insufficient buying power
ZG 0.6238326
An error occured attempting to buy 20.0 shares of ZG insufficient buying power
TFC 0
STT 0.714767
An error occured attempting to buy 18.0 shares of STT insufficient buying power
SWKS 0.6580433
An error occured attempting to buy 9.0 shares of SWKS insufficient buying power
NVO 0.5596752
An error occured attempting to buy 15.0 shares of NVO insufficient buying power
AU 0.434106
FTV 0.6889916
KBH 0.5953431
An error occured attempting to buy 40.0 shares of KBH insufficient buying power
KDP 0.5813333
An error occured attempting to buy 38.0 shares of KDP insufficient buying power
SINA 0.6032162
An error occured attempting to buy 29.0 shares of SINA insufficient buying power
DLPH 0.61096877
An error occured attempting

In [19]:
for symbol in tqdm.tqdm(symbols):
    print(symbol in open_orders)

NameError: name 'open_orders' is not defined